# 11.策略梯度算法
>Q-learning、DQN 及 DQN 改进算法都是 **基于价值（value-based）** 的方法，在强化学习中，除了基于值函数的方法，还有一支非常经典的方法：**基于策略（policy-based）** 的方法。

- **基于值函数的方法** 主要是学习值函数，然后根据值函数导出一个策略，学习过程中并不存在一个显式的策略
- **基于策略的方法** 则是直接显式地学习一个目标策略

其中 **策略梯度** 是基于策略的方法的基础。